In [338]:
import json

In [339]:
# get discussion_list
try:
    cache_file = open('discussion_list3.json', 'r')
    cache_contents = cache_file.read()
    discussion_lists = json.loads(cache_contents)
    cache_file.close()
except:
    print("something bad happens!")
    
try:
    cache_file = open('leetcode_questions.json', 'r')
    cache_contents = cache_file.read()
    leetcode_questions = json.loads(cache_contents)
    cache_file.close()
except:
    print("something bad happens!")
    
try:
    cache_file = open('problem_descriptions.json', 'r')
    cache_contents = cache_file.read()
    problem_descriptions = json.loads(cache_contents)
    cache_file.close()
except:
    print("something bad happens!")

# BM25-CTF Algorithm

In [579]:
import pickle
def savePickle(dictfile,filename):
	print("saving....")
	f=open(filename+".pickle",'wb')
	pickle.dump(dictfile,f)
	f.close()

def loadPickle(filename):
	print("loading....")
	f=open(filename+".pickle",'rb')
	dictfile=pickle.load(f)
	f.close()
	return dictfile

In [580]:
meta=loadPickle('meta_stop')

loading....


In [590]:
weighted_scores=loadPickle('weighted_scores')

loading....


In [342]:
f=open('documents_id.txt')
lines=f.readlines()
f.close()
document_map={}
for each in lines:
    my_data=each.split("\t")
    document_map[my_data[0]]=my_data[1][:-1]

In [343]:
import math

class bm25_ctf():                                            
                                                                       
    def __init__(self, k1 = 1.25, b = 0.9, k3 = 500):                                             
        self.k1 = k1
        self.b = b
        self.k3 = k3                                      
                                                                                 
    def score_one(self, w,query_meta,meta,document_id):
        k1 = self.k1
        b = self.b
        k3 = self.k3
        #Fill your answer here
        ictf=math.log(meta['total_terms']/meta['corpus_term_count'][w])
        pidf=math.log(-(1-2**(-meta['corpus_term_count'][w]/meta['num_docs']))/(math.log(1-meta['doc_count'][w]/meta['num_docs']))+1)
        idf=math.log((meta['num_docs']-meta['doc_count'][w]+0.5)/(meta['doc_count'][w]+0.5))
        bidf=ictf*pidf*idf
        tf=((k1+1)*meta['btf'][w][document_id])/(k1*(1-b+b*meta['doc_size'][document_id]/meta['avg_dl'])+meta['btf'][w][document_id])
        qtf=((k3+1)*query_meta['query_term_weight'][w])/(k3+query_meta['query_term_weight'][w])
        
        res=bidf*tf*qtf
        
        return res
    
    def score(self,query_meta,document_id,meta):
        words=set(query_meta['query_words']) & meta['doc_unique_terms'][document_id]
        score=0
        for w in words:
            score+=self.score_one(w,query_meta,meta,document_id)
        return score

In [600]:
def get_weighted_score(query_meta,i,weighted_scores):
    score=0
    prog_lan_score=weighted_scores['prog_lan_score']
    tag_score=weighted_scores['tag_score']
    difficulty_score=weighted_scores['difficulty_score']
    title_score=weighted_scores['title_score']
    keywords_in_discussion=weighted_scores['keywords_in_discussion']
    for t in tag_score:
        if t in query_meta['query']:
            if str(i) in tag_score[t]:
                score+=tag_score[t][str(i)]*0.1
        if t in query_meta['query']:
            if str(i) in keywords_in_discussion[t]:
                score+=keywords_in_discussion[t][str(i)]
    for t in title_score:
        if t in query_meta['query']:
            if str(i) in title_score[t]:
                score+=title_score[t][str(i)]

    for w in query_meta['query'].split():
        if w in weighted_scores['prog_lan_score']:
            if i in weighted_scores['prog_lan_score'][w]:
                score+=weighted_scores['prog_lan_score'][w][i]
        if w in weighted_scores['difficulty_score']:
            if str(i) in weighted_scores['difficulty_score'][w]:
                score+=weighted_scores['difficulty_score'][w][str(i)]
        elif w=='difficult':
            if str(i) in weighted_scores['difficulty_score']['hard']:
                score+=weighted_scores['difficulty_score']['hard'][str(i)]
    
    vote_score=weighted_scores['vote_score']
    view_score=weighted_scores['view_score']
    if str(i) in vote_score:
        score+=vote_score[str(i)]*0.01+view_score[str(i)]*0.0001
                
    return score

In [618]:
def get_query_meta(query):
    if 'dynamic programming' in query:
        query+=' dp'
    elif 'dp' in query:
        query+=' dynamic programming '
        
    if query in query_logs:
        return query_logs[query]
    else:
        query_words=query.lower().split()
              
        query_length=len(query_words)
        query_term_weight={}
        for qw in query_words:
            if qw not in query_term_weight:
                query_term_weight[qw]=1
            else:
                query_term_weight[qw]+=1
        

        query_meta={}
        query_meta['query_length']=query_length
        query_meta['query_term_weight']=query_term_weight
        query_meta['query_words']=query_words
        query_meta['query']=query
        query_meta['related_query']=[]
        query_logs[query]=query_meta
        return query_meta

In [619]:
import pandas as pd
def do_search(query,meta,weighted_scores):
    d_scores={}
    query_meta=get_query_meta(query)
    print("query:",query_meta['query'])
    print("result:")
    for i in range(1,meta['num_docs']):      
        d_scores[i]=[ranker.score(query_meta,i,meta)+get_weighted_score(query_meta,i,weighted_scores)]
    my_frame=pd.DataFrame.from_dict(d_scores).T
    my_frame.columns=['scores']
    my_frame=my_frame.sort_values(by='scores',ascending=False)
    print(my_frame.head(10))
#     print("related query:")
#     RQ_len=5
#     if len(query_meta['related_query'])<5:
#         RQ_len=len(query_meta['related_query'])       
#     for i in range(RQ_len):
#         print(query_meta['related_query'][i], end=" ")
    return d_scores,my_frame

In [620]:
query_logs={}

In [621]:
ranker=bm25_ctf()

In [622]:
query="difficult question with dynamic programming"

In [623]:
d_scores,my_frame=do_search(query,meta,weighted_scores)

query: difficult question with dynamic programming dp
result:
          scores
1262  222.278301
1990  200.550230
36    142.341665
529   141.444494
566   140.646702
181   140.145225
611   139.456499
567   138.108750
846   137.966358
131   137.754428


In [624]:
id_=list(my_frame.head(10).index.values)
id_

[1262, 1990, 36, 529, 566, 181, 611, 567, 846, 131]

In [625]:
for each in id_:
    qid,did=document_map[str(each)].split("-")
    print(discussion_lists[str(qid)][int(did)]['discussion_title'])
    print(discussion_lists[str(qid)][int(did)]['discussion_link'])
#     print(leetcode_questions[str(qid)]['problem_title'])
#     print(leetcode_questions[str(qid)]['problem_url'])
    
#     print(leetcode_questions[str(qid)])
#     print(discussion_lists[str(qid)][int(did)])

C++ dynamic programming, O(N^3), 32 ms, with comments
https://leetcode.com/problems/burst-balloons/discuss/76232/C%2B%2B-dynamic-programming-O(N3)-32-ms-with-comments
Why does Greedy solution work for this; why not Dynamic Programming ?
https://leetcode.com/problems/ipo/discuss/142490/Why-does-Greedy-solution-work-for-this-why-not-Dynamic-Programming
Easy DP Java Solution with detailed Explanation
https://leetcode.com/problems/regular-expression-matching/discuss/5651/Easy-DP-Java-Solution-with-detailed-Explanation
7-10 lines C++ Solutions with Detailed Explanations (O(m*n) time and O(m) space)
https://leetcode.com/problems/distinct-subsequences/discuss/37316/7-10-lines-C%2B%2B-Solutions-with-Detailed-Explanations-(O(m*n)-time-and-O(m)-space)
Is it Best Solution with O(n), O(1).
https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/discuss/39611/Is-it-Best-Solution-with-O(n)-O(1).
Linear runtime and constant space solution
https://leetcode.com/problems/wildcard-matching/disc

# add some feedback

In [1]:
def get_update_word_list(item_list,threshold=3):
    word_list=[]
    for i in item_list:
        for w in meta['doc_unique_terms'][i]:
            important_score=meta['btf'][w][i]*(1/meta['doc_count'][w])
            if important_score>threshold:
                word_list.append(w)
                print(w,important_score)
    return word_list

In [2]:
# used for feedback
def query_meta_update(query_meta,word_lists,update_rate=1):
    
    for w in word_lists:
        query_meta['query_words'].append(w)
        if w not in query_meta['query_term_weight']:
            query_meta['query_term_weight'][w]=update_rate
        else:
            query_meta['query_term_weight'][w]+=update_rate
        query_meta['query_length']+=update_rate*len(word_lists)
        
        query_meta['related_query'].append(w)
    
    return query_meta

In [259]:
word_list=get_update_word_list(item_list)

good 4.0
2018 4.0
haha 4.0
luck 4.0


In [260]:
query_logs[query]=query_meta_update(get_query_meta(query),word_list)

In [261]:
d_scores=do_search(query,meta)

query: hello baby * good 2018 haha luck * 
result:
0 0.37061128130176607
1 0
2 0.1938100058015275
3 2.2973177085906604
4 9.189270834362642
related query:
hello baby 